In [ ]:
## Building Machine Learning Pipeline : Data Analysis Phase

#In this we will focus on creating Machine Learning Pipelines considering all the life cycle of a DataScience Projects.

# Project name : House Price : Advanced Regession Technique

The main aim of this project is to predict the house price based on various features which we will discuss as we go ahead

Dataset To be Download Below Link :

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

### All the Lifecycle In A Data Science Projects
Data Analysis

Feature Engineering

Feature Selection

Model Building

Model Deployment

## Data Analysis Phase
## MAin aim is to understand more about the data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
## Display all the columns of the dataframe

pd.pandas.set_option('display.max_columns',None)

dataset=pd.read_csv('train.csv')

## print shape of dataset with rows and columns
print(dataset.shape)

## print the top5 records
dataset.head()

#### #### In Data Analysis We will Analyze To Find out the below stuff
1. Missing Values
2. All The Numerical Variables
3. Distribution of the Numerical Variables
4. Categorical Variables
5. Cardinality of Categorical Variables
6. Outliers
7. Relationship between independent and dependent feature(SalePrice)


## Missing Values :

## Here we will check the percentage of nan values present in each feature
features_with_na=[features for features in dataset.columns if dataset[features].isnull().sum()>1]

for feature in features_with_na:
    print(feature, np.round(dataset[feature].isnull().mean(), 3),  ' % Missing Values')

##### Since There Are many Missing Values , We need To find the Reletionship between Missin values and Sales Price


# let's Plot soe Daigram for The Relestionship

for feature in features_with_na:
    data = dataset.copy()
    
    data[feature] = np.where(data[feature].isnull(), 1, 0)
    
    # calculate the mean saleprice where the information is missing
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.title(feature)
    plt.show()

Note : Here We Clearly See The Reletionship Between Missing Vslues and Dependent Variable. So we need to replace missing values with some meaninngfull var which we do in feature Engineering section.


## Numerical Variables

numerical_features = [feature for feature in dataset.columns if dataset[feature].dtypes != 'O']
print('Number of numerical variable : ', len(numerical_features))

#visualize the numerical variables
dataset[numerical_features].head()

Temoproal Variable (Datetime variable)

# List of Variable contain year information
year_feature = [feature for feature in numerical_features if 'Yr' in feature or 'Year' in feature ]
year_feature

## Lets analyze the Temporal Datetime Variables
## We will check whether there is a relation between year the house is sold and the sales price

dataset.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('Year Sold')
plt.ylabel('Median House Price')
plt.title("House Price vs Year Sold")

## Hear we will compare the difference between all years feature with SalePrice

for feature in year_feature:
    if feature!='YrSold':
        data=dataset.copy()
        ## We will capture the difference between year variable and year the house was sold for
        data[feature]=data['YrSold']-data[feature]

        plt.scatter(data[feature],data['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalePrice')
        plt.show()


## Numerical variables are usually of 2 type
## 1. Continous variable and Discrete Variables

discrete_feature=[feature for feature in numerical_features if len(dataset[feature].unique())<25 and feature not in year_feature+['Id']]
print("Discrete Variables Count: {}".format(len(discrete_feature)))

discrete_feature

## Lets Find the realtionship between them and Sale PRice

for feature in discrete_feature:
    data=dataset.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

### Continuous Variable

continuous_feature=[feature for feature in numerical_features if feature not in discrete_feature+year_feature+['Id']]
print("Continuous feature Count {}".format(len(continuous_feature)))

## Lets analyse the continuous values by creating histograms to understand the distribution

for feature in continuous_feature:
    data=dataset.copy()
    data[feature].hist(bins=25)
    plt.xlabel(feature)
    plt.ylabel("Count")
    plt.title(feature)
    plt.show()




## we will be using Logarithmic Transformation

for feature in continuous_feature:
    data=dataset.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        data['SalePrice']=np.log(data['SalePrice'])
        plt.scatter(data[feature],data['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalesPrice')
        plt.title(feature)
        plt.show()

## outliers

for feature in continuous_feature:
    data=dataset.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        data.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()
        
    

## Categorical Variables

categorical_features=[feature for feature in dataset.columns if data[feature].dtypes=='O']
categorical_features

dataset[categorical_features].head()

for feature in categorical_features:
    print('The feature is {} and number of categories are {}'.format(feature,len(dataset[feature].unique())))

## Find out the relationship between categorical variable and dependent feature SalesPrice

for feature in categorical_features:
    data=dataset.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

# Advanced Housing Prices - Feature Engineering

The Main aim of the poject is to predict house price based on given features.

dataset To be downloaded from the below link

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

We will be performing all the below steps in Feature Engineering

Missing values
Temporal variables
Categorical variables: remove rare labels
Standarise the values of the variables to the same range

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# to visualize all the columnns inn the dataframe
pd.pandas.set_option('display.max_columns', None)

## Merge Train And Test dataset Fo Feature Enginering And Scaling
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
dataset_mrg = pd.concat([train_df,test_df])

dataset_mrg.shape

dataset = pd.DataFrame(dataset_mrg)

dataset.head()
dataset.shape





## Create Data Splitting in train and test data 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(dataset,dataset['SalePrice'],test_size=0.1,random_state=0)

X_train.shape, X_test.shape

### Missing Values

features_nan = [feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes=='O']

for  feature in features_nan:
    print("{}:{}% missing values".format(feature,np.round(dataset[feature].isnull().mean(),3)))

## Replace missing value with a new label
def replace_cat_feature(dataset,features_nan):
    data=dataset.copy()
    data[features_nan]=data[features_nan].fillna('Missing')
    return data

dataset=replace_cat_feature(dataset,features_nan)

dataset[features_nan].isnull().sum()

dataset.head(10)

## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(dataset[feature].isnull().mean(),3)))

## Replacing the numerical Missing Values

for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=dataset[feature].median()
    #print(median_value)
    
 ## create a new feature to capture nan values
    dataset[feature+'_nan']=np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(median_value,inplace=True)
    
dataset[numerical_with_nan].isnull().sum()
    

dataset.head(8)

## Temporal Variables (Date Time Variables)

for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:
    dataset[feature]=dataset['YrSold']-dataset[feature]

dataset.head()

dataset[['YearBuilt','YearRemodAdd','GarageYrBlt']].head()

import numpy as np
num_features=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']

for feature in num_features:
    dataset[feature]=np.log(dataset[feature])

dataset.head()

## Handlinng Rare Categorical Feature

we will remove categorical variabe that are present less then 1% of the Observations

categorical_features=[feature for feature in dataset.columns if dataset[feature].dtype=='O']

categorical_features

for feature in categorical_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.01].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')
    
    

dataset.head(50)

scaling_feature=[feature for feature in dataset.columns if feature not in ['Id','SalePerice'] ]
len(scaling_feature)

scaling_feature



for feature in categorical_features:
    labels_ordered=dataset.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    dataset[feature]=dataset[feature].map(labels_ordered)
    

dataset.head()

# Feature Scaling

feature_scale=[feature for feature in dataset.columns if feature not in ['Id','SalePrice']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(dataset[feature_scale])

scaler.transform(dataset[feature_scale])

# transform the train and test set, and add on the Id and SalePrice variables
data = pd.concat([dataset[['Id', 'SalePrice']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(dataset[feature_scale]), columns=feature_scale)],
                    axis=1)

data.head()

data.to_csv('new_dataset.csv',index=False)

# Feature Selection Advanced House Price Prediction

The main aim of project is to predict the house price based on various features

### Dataset to be Downloaded from the below link:


https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## for feature selection
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualize all the columns in the dataframe
pd.pandas.set_option('display.max_columns',None)

dataset=pd.read_csv("new_dataset.csv")


dataset=dataset.drop_duplicates()
dataset = dataset.drop(['SalePrice_nan'],axis=1)
dataset.head(2122)

np.any(np.isnan(dataset))

np.all(np.isfinite(dataset))

np.where(np.isnan(dataset))

dataset =dataset.fillna(dataset.mean())

 # set dependent variable
y_train =dataset[['SalePrice']]

y_train.head()

## drop dependent feature from dataset
X_train=dataset.drop(['Id','SalePrice'],axis=1)
X_train.head()



### Apply Feature Selection
# first, I specify the Lasso Regression model, and I
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

# Then I use the selectFromModel object from sklearn, which
# will select the features which coefficients are non-zero

feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=0))
feature_sel_model.fit(X_train, y_train)

feature_sel_model.get_support()

# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feat = X_train.columns[(feature_sel_model.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))


selected_feat

X_train=X_train[selected_feat]
X_train.head()

X_train.shape



y_train.to_csv('y_train')

X_train.to_csv('X_train')

# Reression Model : Advance House Price Prediction

After completing Featuer Engineering and Feature selection , Now We Create Regression Model TO Predict House Price.

## Import Libraiers
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error as MSE 

# import dataset train And Test
X = pd.read_csv('X_train')
y = pd.read_csv('y_train')

y

y = y.drop(['Unnamed: 0'],axis=1)
X= X.drop(['Unnamed: 0'],axis=1)

print(X.shape)
print(y.shape)

X.head()

## Train_Test_split Dataset

## Create Data Splitting in train and test data 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.6,random_state=0)



### Prediction And Selecting Alorithem

import xgboost
regressor=xgboost.XGBRegressor()

booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

## Hyper Parameter Optimization

n_estimators = [100,500,900,1100,1500]
max_depth = [2,3,5,10,15]
booster = ['gbtree','gblinear']
learning_rate = [0.05, 0.1, 0.15, 0.20]
min_child_weight=[1,2,3,4]

# Define Hyperparameter to Search
hyperparameter_grid ={
     'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
}

# Set up the random search with 4-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

random_cv.fit(X_train,y_train)

random_cv.best_estimator_

regressor=xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=15, min_child_weight=4, missing=None, n_estimators=900,
       n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

regressor.fit(X_train,y_train)

import pickle
filename = 'Ad_House_price_model.pkl'
pickle.dump(regressor, open(filename, 'wb'))

# Predict the model 
y_pred = regressor.predict(X_test) 

y_pred

##Create Sample Submission file and Submit using ANN
pred=pd.DataFrame(y_pred)
sub_df=pd.read_csv('sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred],axis=1)
datasets.columns=['Id','SalePrice']
datasets.to_csv('sample_submission.csv',index=False)





### Completed####